<a href="https://colab.research.google.com/github/superduduguan/leetcode/blob/master/leetcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EASY

## 20.Valid Parentheses(Stack)
Given a string containing just the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid.

###栈

In [0]:
class Solution:
  def isValid(self, s: str) -> bool:
    stack = []
    dict = {'(':')', '[':']', '{':'}'}
    for n in s: #重要！这句话一旦写下就确定了n的取值，后面不再变了！！！
      if n in dict:
        stack.append(n)
      elif stack:
        pop = stack.pop()
        if n != dict[pop]:
          return False
      else:
        return False
    return stack == []

##26.remove-duplicates-from-sorted-array
Given a sorted array nums, remove the duplicates in-place such that each element appear only once and return the new length.

###双指针 
快指针遇到不重复的就把新元素放到慢指针处
由于慢指针没有完全遍历，所以相当于只循环了一轮

In [0]:
class Solution: #双指针原地从头覆盖~O(n)
  def removeDuplicates(self, nums):
    i = 1 #不管是什么情况，第一个数肯定是保留的
    count = 1
    if not nums:
      return 0
    for index in range(1, len(nums)):# 当range前后界相等时，不进入循环！通过这个特性避免len=1无法比较的情况！
      if nums[index] != nums[index - 1]:
        nums[i] = nums[index]
        i += 1
        count += 1
    return count

###pop 
遇到重复的就弹出

In [0]:
class Solution: #pop~O(n^2)
  def removeDuplicates(self, nums):
    count = 1 #对待初始状态要认真
    index = 0
    if len(nums) == 0: #面对比较（牵扯两个元素），要先考虑空集情况
      return 0
    while True:
      if index == len(nums)-1: #首先应考虑到 len = 1的情况，在循环的开始阶段使用弹出控制会有奇效
        return count
      if nums[index] == nums[index + 1]:
        nums.pop(index) #注意pop之后要不要改变指针位置
      else:
        count += 1
        index += 1

##53.Maximum Subarray
Given an integer array nums, find the contiguous subarray (containing at least one number) which has the largest sum and return its sum.

###暴力枚举
包含大量重复运算，比如在计算sum（1，5）和sum（1，6）时，相加的指针重复遍历了两次（1，5），共移动9个位置

In [0]:
class Solution: #~O(n^3)
    def maxSubArray(self, nums):
        SUM = []
        for i in range(len(nums)):
            for j in range(i+1, len(nums)):
                sum = 0
                for k in range(i, j+1):
                    sum += nums[k]
                print(i,j,sum)
                SUM.append(sum)
        for i in range(len(nums)):
            SUM.append(nums[i])
        print(SUM)
        return max(SUM)


###优化暴力枚举
&emsp;&emsp;求连续和的时候，边加边保存（比较）可以用第二轮循环就完成  
&emsp;&emsp;与暴力枚举法的区别在于，对于每个窗，不需要指针不停返回初始位置，减少了大量重复运算。比如在计算sum（1，5）和sum（1，6）时，相加的指针只需移动一次  
&emsp;&emsp;另外，每得到一个值就进行两两比较，节省空间

In [0]:
class Solution: #~O（n^2）
    def maxSubArray(self, nums: List[int]) -> int:
        n = len(nums)#好习惯
        maxSum = nums[0]#好习惯
        sum = 0
        
        for i in range(n):
            sum = 0
            for j in range(i, n):
                sum += nums[j]
                maxSum = max(maxSum, sum)#！随时比较
        
        return maxSum 

###前序和
利用sum(a,b)=presum(b)-presum(a-1)  
进一步优化时，要求maxsum，上式直接减去0到a-1间最小的presum即可

In [0]:
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        n = len(nums)
        maxSum = nums[0]
        leastSum = 0 #到目前为止所有presum（a-1）中的最小值）
        presum = 0 #前项和

        for i in range(n):
            presum += nums[i] #记录每个b位置的前项和
            maxSum = max(maxSum, presum - leastSum) #一直取最大的maxSum
            leastSum = min(leastSum, presum) #这个leastSum是上一轮的（a-1）的，必须最后才更新 #一直取最小
            
        return maxSum

###！动态规划Kadane's Algorithm
&emsp;&emsp;index从0遍历到n-1的过程中，每个元素面临两个选择：要么自己重新开始一个新的序列（此时sum等于自己），要么继续拓展前面最大的序列（此时sum为i-1时刻的sum加自己）。  
&emsp;&emsp;这样两个选择比大小就可知：**从任意元素开始**，**在这个i元素处结束**的话最大的sum是多少  
  
  
设定状态表示量dp[i]：到当前位置 i 为止的最大子序列和  
状态转移方程为： dp[i] = max(dp[i - 1] + nums[i], nums[i])  
初始化：dp[0] = nums[0]


In [0]:
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        n = len(nums)
        max_global = nums[0]
        max_end_here = 0
        for i in range(n):
            max_end_here = max(nums[i], nums[i] + max_end_here) #重要！这种递归，括号里的max时上一轮的！
            max_global = max(max_end_here, max_global) #重要！这种表示，直接理解为求max_end_here的最大值
        return max_global

&emsp;&emsp;另一种理解：遍历到的每个元素：如果有黑历史（之前sum为负），则重新开始；如果有历史功绩（之前sum为正），则继续延续    
&emsp;&emsp;与前一种理解相比：前者用nums[i]和nums[i] + max_end_here相比；后者用max_end_here和0相比，只有当大于时才进行nums[i] + max_end_here，赋值运算更少

In [0]:
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        n = len(nums)
        max_global = nums[0]
        max_end_here = 0
        for i in range(n):
            if max_end_here > 0:
                max_end_here += nums[i]
            else:
                max_end_here = nums[i]
            max_global = max(max_global, max_end_here)
        return max_global


&emsp;&emsp;进一步优化：对于这种只需要遍历一次的队列，可充分利用已遍历过的元素,用nums[i]表示当前元素，nums[i-1]代表之前的maxsum。这样在maxsum<0需要重开新序列时，无需赋值，nums[i]就是新的nums[i-1]

In [0]:
i-1class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        n = len(nums)
        for i in range(1, n):
            if nums[i-1] > 0:
                nums[i] += nums[i-1]
        return max(nums)


###分治法

最大和子序列出现位置有三种可能：左边、右边、跨过mid。三种情况的最大值即为答案。  
其中跨过mid的max = 以mid-1结尾的max + 以mid+1开头的max + mid  
（慢）

In [0]:
class Solution: ~O(nlogn) #涉及二分法通常是nlogn
    def maxSubArray(self, nums):
        return self.helper(nums, 0, len(nums) - 1)
    
    def helper(self, nums, left, right): #常见，分治法辅助函数包含左右端点
        if left >= right:
            return nums[left]

        mid = (left + right) // 2
        sum_max_left = self.helper(nums, left, mid - 1) ##一定注意这里左边界不是nums最右端！
        sum_max_right = self.helper(nums, mid + 1, right) #一定注意这里右边界不是nums最右端！

        sum_max_acrossmid_right = 0
        sum = 0
        for i in range(mid + 1, right + 1):
            sum += nums[i]
            sum_max_acrossmid_right = max(sum_max_acrossmid_right, sum)
        
        sum_max_acrossmid_left = 0
        sum = 0
        for i in reversed(range(left, mid)): ##相当于range(mid-1, left-1, -1),因为包前不包后
            sum += nums[i]
            sum_max_acrossmid_left = max(sum_max_acrossmid_left, sum)
        
        sum_max_acrossmid = sum_max_acrossmid_right + sum_max_acrossmid_left + nums[mid]
        return max(sum_max_left, sum_max_right, sum_max_acrossmid)


## 70.Climbing Stairs
You are climbing a stair case. It takes n steps to reach to the top.

Each time you can either climb 1 or 2 steps. In how many distinct ways can you climb to the top?

Note: Given n will be a positive integer.

###递归
与动态规划的区别在于，递归多次计算了重复的数。动态规划将计算过的数保存了下来后面还可以继续使用。  
即：递归是函数的嵌套，动态规划是用数组记录之前的函数值

In [0]:
class Solution:
    def climbStairs(self, n: int) -> int:
        if n == 1:
            return 1
        if n == 2:
            return 2
        return self.climbStairs(n-1) + self.climbStairs(n-2)

###动态规划
显然是计算总数的动态规划问题，最后一步可以是1，也可以是2 
f(n)表示到达f(n)的路径数  
初始条件为f(1) = 1, f(2) = 2  
转移方程为f(n) = f(n-1) + f(n-2)

In [0]:
class Solution:
    def climbStairs(self, n: int) -> int:
        dict = {}
        dict[1] = 1
        dict[2] = 2
        for i in range(3, n+1):
            dict[i] = dict[i-1] + dict[i-2]
        return dict[n]

## 88.Merge Sorted Array

Given two sorted integer arrays nums1 and nums2, merge nums2 into nums1 as one sorted array.

###双指针
题目要求不占用新的内存空间，则直接将两个list从后向前遍历，比出的大小在m+n处从后向前排列

In [0]:
class Solution:
    def merge(self, nums1, m, nums2, n) -> None:
        k = 1
        i = m - 1
        j = n - 1
        
        while i >= 0 and j >= 0: #while中意思理解为：让i和j之一先下0。如果是j先下0，则i放在原地即可不用管；如果是i先下0，j把剩余的平移到i即可
            if nums1[i] > nums2[j]:
                nums1[m + n - k] = nums1[i]
                i -= 1
            else:
                nums1[m + n - k] = nums2[j]
                j -= 1
            k += 1
            
        if j >= 0:
            nums1[0 : j+1] = nums2[0 : j+1] #重要！数组赋值一定要点对点 https://www.cnblogs.com/monkey-moon/p/9347505.html

## 104.Maximum Depth of Binary Tree
Given a binary tree, find its maximum depth.

The maximum depth is the number of nodes along the longest path from the root node down to the farthest leaf node.

###递归法
由于树是一种递归的数据结构，因此用递归去解决的时候往往非常容易

In [0]:
class TreeNode:
    def __init__(self, x): 
        self.val = x
        self.left = None
        self.right = None

class Solution:
    def maxDepth(self, root: TreeNode) -> int:
        if not root: #写递归一定先把最后情况写出
            return 0
        return 1 + max(self.maxDepth(root.left),self.maxDepth(root.right))#最大层数=左右子树最大层数的大者+本层  #递归一定注意self的用法

###！队列&BFS
广度优先搜索使用队列来实现（1234567子树后进后出），每次遍历都把同层结点遍历完了才进入下一层，可称为层次搜索，很符合题意：  
1、把根节点放到队列的末尾。  
2、每次从队列的头部取出一个元素，查看这个元素所有的下一级元素，把它们放到队列的末尾。并把这个元素记为它下一级元素的前驱。


In [0]:
class Solution(object):
    def maxDepth(self, root):#注意，这里输入的只是根节点
        if not root:
            return 0
        queue = [root]
        print(len(queue))
        depth = 0
        while queue:
            depth += 1
            for i in range(len(queue)):#保证queue里现有的node全被pop，且子节点加入队尾。每次这个循环结束时，queue里的元素都全是下一层的了
                cur_root = queue.pop(0)
                if cur_root.left:
                    queue.append(cur_root.left)
                if cur_root.right:
                    queue.append(cur_root.right)
        return depth


####栈&DFS
深度优先搜索用栈来实现（1245367子树后进先出），每次遍历都要把一条路走到底才再进入同层另一个节点:   
1、把根节点压入栈中。    
2、每次从栈中弹出一个元素，搜索所有在它下一级的元素，把这些元素压入栈中。并把这个元素记为它下一级元素的前驱。



## 121.Best Time to Buy and Sell Stock
Say you have an array for which the $i^{th}$ element is the price of a given stock on day $i$.

If you were only permitted to complete at most one transaction (i.e., buy one and sell one share of the stock), design an algorithm to find the maximum profit.

###暴力枚举

和53题暴力枚举法一样，下面这种解法在每次比较的时候都重复比较了很多元素。在编程时一定主要静态比较重复造轮子

In [0]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n = len(prices)
        benefit = 0
        if n <= 1:
            return 0
        for i in range(1, n):
            buy = min(prices[:i])
            sell = max(prices[i:])
            benefit = max(sell - buy,benefit)
            print(buy, sell, benefit)
        return benefit

###！动态规划Kadane's Algorithm
由于是先买入后卖出，从左向右遍历，保留当下已遍历到的最小buy（一定是当下此时此刻的最优购入时机；历史上的更优profit也一定出自于那时那刻的最小buy），所以一定以profit的最大作为衡量标准（其实就是在比较多个从最波谷到波峰的距离） 
思考这种问题一定不要有全局的静态意识，要随着从左到右一步一步深入

简言之，最大值一定出现在max-min，只需比较不同的max-min即可(遍历的时候，用每个值x-min比较也行)

In [0]:
def maxProfit(prices):
    max_profit, min_price = 0, prices[0]
    for price in prices:
        min_price = min(min_price, price)
        max_profit = max(max_profit, price - min_price)
    return max_profit

## 122.best-time-to-buy-and-sell-stock2
Say you have an array for which the ith element is the price of a given stock on day i.

Design an algorithm to find the maximum profit. You may complete as many transactions as you like (i.e., buy one and sell one share of the stock multiple times).
You may not engage in multiple transactions at the same time (i.e., you must sell the stock before you buy again).

In [0]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        sum = 0
        for i in range(len(prices) - 1):
            if prices[i] < prices[i+1]:
                sum += prices[i+1] - prices[i]
        return sum

## 125.valid-palindrome
Given a string, determine if it is a palindrome, considering only alphanumeric characters and ignoring cases.

Note: For the purpose of this problem, we define empty string as valid palindrome.

In [0]:
class Solution: #使用pop涉及了数组元素的重新分配；此类直接用双指针，只判断不赋值更好
    def isPalindrome(self, s: str) -> bool:
        s = s.translate(str.maketrans('', '', string.punctuation+' ')).lower() #需要注意replace等操作不会改变原string的内容
        s = list(s)
        while len(s) > 1:
            if s.pop(0) != s.pop():
                return 0
        return 1

## 136.Single Number
Given a non-empty array of integers, every element appears twice except for one. Find that single one.

###字典&哈希表
对每个元素计数，最直观的想法是哈希表一次遍历完成，但是多占用了空间

In [0]:
class Solution: #~O(n)
    def singleNumber(self, nums):
        dict = {} #Hashmap用来计数，{num:count}

        for num in nums:
            dict[num] = dict.get(num, 0) + 1  #能找到，就返回key，找不到就返回0

        for key, val in dict.items(): #同时对两个值遍历
            if val == 1:
                return key


###异或运算
由于XOR（^）满足交换律，且两个相同元素异或结果为0，0与单个元素异或结果就是这个单个元素

In [0]:
class Solution:
    def singleNumber(self, nums):
        res = 0
        for num in nums:
            res ^= num
        return res

## 155.Min Stack
Design a stack that supports push, pop, top, and retrieving the minimum element in constant time.

### 暴力遍历

In [0]:
class MinStack: ~O(n)
    def __init__(self):
        self.stack = []

    def push(self, x: int):
        self.stack.append(x)

    def pop(self) -> None:
        return self.stack.pop()

    def top(self) -> int:
        return self.stack[-1]

    def getMin(self) -> int:
        return min(self.stack)

###!双栈同步更新
一个最小栈和原栈同步更新，存储当前最小值，缺点是对空间的耗费

In [0]:
class MinStack: 
    def __init__(self):
        self.stack = []
        self.stack_min = []
        self.Min = float('inf')

    def push(self, x: int):
        self.stack.append(x)
        self.Min = min(self.Min, x)
        self.stack_min.append(self.Min)

    def pop(self):
        self.stack.pop()
        self.stack_min.pop()
        if self.stack_min:#一看到[-1]这种表示，就要立刻警觉空集！
            self.Min = self.stack_min[-1]
        else:
            self.Min = float('inf')

    def top(self) -> int:
        return self.stack[-1]

    def getMin(self) -> int:
        return self.Min

###差值法
栈内压入的是与前一时刻最小值的差值，然后再更新此刻最小值；这样一来，栈顶存储的数字的正负可判断栈顶代表的数字是否为当前最小值。如果栈顶存储的数字为负数，意味着栈顶代表的数字比原来的最小值小，为当前最新的最小值

In [0]:
class MinStack:

    def __init__(self):

        self.Min = float('inf') #储存当前最小值
        self.stack = []

    def push(self, x: int):
        self.stack.append(x - self.Min) #存入与当前最小值的差值
        if x < self.Min: #先压入与之前最小值的差值，再更新此刻最小值；栈顶的正负可判断是否为当前最小值
            self.Min = x

    def pop(self):
        tmp = self.stack.pop()
        if tmp < 0:
            self.Min -= tmp

    def top(self):
        tmp = self.stack[-1]
        if tmp < 0:
            return self.Min
        else:
            return self.Min + tmp

    def getMin(self):
        return self.Min

## 167.Two Sum II - Input array is sorted
Given an array of integers that is already sorted in ascending order, find two numbers such that they add up to a specific target number.  

The function twoSum should return indices of the two numbers such that they add up to the target, where index1 must be less than index2.

###暴力遍历
两轮遍历遍历了全部符合题意的情况，但对流程缺少控制，有大量无用流程

In [0]:
class Solution: 
    def twoSum(self, numbers: List[int], target: int) -> List[int]:
        n = len(numbers)
        for index1 in range(n):
            for index2 in range(index1+1, n):
                if numbers[index1] + numbers[index2] == target:
                    return [index1+1, index2+1]

###双指针
利用双指针的可控制性，从两头向中间遍历，一个指针逐渐变大，一个指针逐渐变小。对于每一个两元素和，要么大了要么小了，每次改变只牵扯一个指针

In [0]:
class Solution:
    def twoSum(self, numbers: List[int], target: int) -> List[int]:
        left, right = 0, len(numbers) - 1
        while left < right:
            if numbers[left] + numbers[right] < target:
                left += 1
            if numbers[left] + numbers[right] > target:
                right -= 1
            if numbers[left] + numbers[right] == target:
                return [left+1, right+1]

###！字典&哈希表
构建字典{val, index + 1}来存储已经遍历过的数，对比暴力法相当于用空间换时间  
在构建字典的时候，常见 ifXXXinXXX,else存入字典

In [0]:
class Solution:
    def twoSum(self, numbers: List[int], target: int) -> List[int]:
        visited = {}
        for index, val in enumerate(numbers): #enumerate用于数组，items用于字典
            if target - val in visited: #当前数所需另一半已被读取过
                return [visited[target-val], index+1]
            else:
                visited[val] = index + 1 #构建字典{val, index + 1}

##169.Majority Element
Given an array of size n, find the majority element. The majority element is the element that appears more than ⌊ n/2 ⌋ times.

You may assume that the array is non-empty and the majority element always exist in the array.


###字典&哈希表
和136题一样，对队列中每个数计数，常用哈希表，时间复杂度为O(1)，但空间复杂度多开了一个字典

In [0]:
class Solution:
    def majorityElement(self, nums: List[int]) -> int:
        dict = {}
        n = len(nums)
        for num in nums:
            dict[num] = dict.get(num, 0) + 1
        for key, val in dict.items():
            if val > n / 2:
                return key

###trick
由于这个数的个数大于2/n，所以在排序后，这些数全部列在一起，一定会经过队列中点，所以排序后中点的数就是所求

In [0]:
class Solution:
    def majorityElement(self, nums):
        nums.sort()
        return nums[len(nums)//2]

##172.Factorial Trailing Zeroes
Given an integer n, return the number of trailing zeroes in n!.

###数论
其实是在求1到n中因子5的个数,这时候运用数论知识，而不应该直接遍历  
$f(n)=\frac{n}{5}+\frac{n}{5^2}+...$

In [0]:
class Solution:
    def trailingZeroes(self, n: int) -> int:
        count = 0
        while n >= 5:
            n = n // 5
            count += n
        return count

##189.Rotate Array
Given an array, rotate the array to the right by k steps, where k is non-negative.

Note:

- Try to come up as many solutions as you can, there are at least 3 different ways to solve this problem.
- Could you do it in-place with O(1) extra space?

###新数组

In [0]:
class Solution:
    def rotate(self, nums: List[int], k: int) -> None:
        queue = []
        n = len(nums)
        k = k % n
        queue[:] = nums[-k:]
        nums[k:] = nums[:-k]
        nums[:k] = queue[:]  

###新数

In [0]:
        while k > 0:
            tmp = nums[-1]
            nums[1:] = nums[:-1]
            nums[0] = tmp
            k -= 1 

###三次翻转法
- 先把[0, n - k - 1]翻转
- 然后把[n - k, n - 1]翻转
- 最后把[0, n - 1]翻转

In [0]:
class Solution: #写函数的时候一定写成（start, end）的形式，不要用数组的切片来表示
    def rotate(self, nums: List[int], k: int) -> None:
        n = len(nums)
        k = k % n
        if k == 0:
            return nums
        self.reverse(nums, 0, n - k - 1)
        self.reverse(nums, n - k, n - 1)
        self.reverse(nums, 0, n - 1)
    def reverse(self, nums , start, end):
        while end > start:
            nums[start], nums[end] = nums[end], nums[start]
            start += 1
            end -= 1
        return nums

class Solution:#如果一定要用数组的切片来表示，一定要用对应的数组切片来接返回的数组！
    def reverse(self, array):
        low = 0 
        high = len(array)-1
        while low < high:
            array[low], array[high] = array[high], array[low]
            low += 1
            high -= 1
        return array
    
    def rotate(self, nums: List[int], k: int) -> None:
        k = k%len(nums)
        nums[-k:] = self.reverse(nums[-k:])###############
        nums[:-k] = self.reverse(nums[:-k])###############
        self.reverse(nums)

##190.Reverse Bits
Reverse bits of a given 32 bits unsigned integer.

###！基础位操作
类似于queue，想象成不停地把队尾拿出来放入一个新队列


In [0]:
class Solution:
    def reverseBits(self, n):
        result = 0                  
        for i in range(32):
            result = result << 1        #类似于queue，整体移位，空出来的地方补0，最先排进队尾的最后会变为队首
            result = result + (n & 1)     #把内容放到队尾，n&1意思是取最后一位
            n = n >> 1             #取n的倒数第i位
        return result

## 191.Number of 1 Bits
Write a function that takes an unsigned integer and return the number of '1' bits it has (also known as the Hamming weight).

###基础位操作

In [0]:
class Solution:
    def hammingWeight(self, n: int) -> int:
        count = 0
        for i in range(32):
            if n & 1 == 1:
                count += 1
            n = n >> 1
        return count

###trick
因为n-1就相当于把n中的最后一个1变为0，这个1后面的所有0变为1  
所以每运行一次n = n & (n - 1)就可以消除这个末尾的1，同时这个1之后依然为0  
 


In [0]:
class Solution(object):
    def hammingWeight(self, n):
        count = 0
        while n:
            n = n & (n - 1)
            count += 1
        return count

## @198.House Robber
You are a professional robber planning to rob houses along a street. Each house has a certain amount of money stashed, the only constraint stopping you from robbing each of them is that adjacent houses have security system connected and it will automatically contact the police if two adjacent houses were broken into on the same night.

Given a list of non-negative integers representing the amount of money of each house, determine the maximum amount **of** money you can rob tonight without alerting the police.

###动态规划

##203.Remove Linked List Elements
Remove all elements from a linked list of integers that have value val.


做链表题：curve！dummy head！(用next即可遍历全部元素)

In [0]:
# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, x):
#         self.val = x
#         self.next = None

class Solution:
    def removeElements(self, head: ListNode, val: int) -> ListNode: #对链表进行操作时，都用头指针来表示
        prev = ListNode(0) #新初始化一个dummy head并赋值
        prev.next = head #为dummy head指定下一个元素
        cur = prev #指针放在dummy head
        while cur.next:
            if cur.next.val == val:
                cur.next = cur.next.next
            else:
                cur = cur.next
        return prev.next

##背过206.Reverse Linked List
Reverse a singly linked list.(utility problem)

###双指针迭代
核心是把每一步中的cur.next赋值为prev，把向右的箭头改为向左。所以需要cur表示现在，prev表示之前。但将cur.next赋值为prev之后胡就无法表示下一个元素了，所以一开始就用nex来备份

In [0]:
# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, x):
#         self.val = x
#         self.next = None

class Solution:
    def reverseList(self, head: ListNode) -> ListNode:
        #initialization
        cur = head
        prev = None
        while cur:
            #backup
            nex = cur.next
           
            #reverse
            cur.next = prev#注意不要写反了
            
            #forward
            prev = cur
            cur = nex
            
        return prev #本来应该是cur，但最后一步把cur赋值成了nex了

###递归
用递归函数把以head.next开头的子链翻转，再把head.next到head,以及head到None的箭头翻转，返回子链的开头即可

In [0]:
class Solution:
    def reverseList(self, head: ListNode) -> ListNode:
        if not head or not head.next: return head
        ans = self.reverseList(head.next)
        head.next.next = head
        head.next = None
        return ans

##219.Contains Duplicate II
Given an array of integers and an integer k, find out whether there are two distinct indices i and j in the array such that nums[i] = nums[j] and the absolute difference between i and j is at most k.

###字典&哈希表
哈希表不仅可以储存出现次数，也可以储存之前出现数字的index。在不限制存储空间的情况下应当优先想到哈希表

In [0]:
class Solution:
    def containsNearbyDuplicate(self, nums: List[int], k: int) -> bool:
        dict = {}
        for index, num in enumerate(nums): #常用！记住！
            if num in dict and index - dict.get(num) <= k:
                return True
            dict[num] = index
        return False


##226.Invert Binary Tree

###递归
看到树的递归结构，直接上递归

由于每个节点只遍历了一次，时间复杂度为O(n)

In [0]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution:
    def invertTree(self, root: TreeNode) -> TreeNode:
        
        if root and (root.left or root.right):
            root.left, root.right = self.invertTree(root.right), self.invertTree(root.left)
        return root

##232.Implement Queue using Stacks
一定注意！在init模块中定义的量，在其他模块引用要加self

In [0]:
class MyQueue:

    def __init__(self):
        self.queue = []

    def push(self, x: int) -> None:
        self.queue.append(x)

    def pop(self) -> int:
        newqueue = []
        while self.queue:
            newqueue.append(self.queue.pop())
        s = newqueue.pop()
        while newqueue:
            self.queue.append(newqueue.pop())
        return s

    def peek(self) -> int:
        return self.queue[0]

    def empty(self) -> bool:
        return not self.queue


##263.Ugly Number
Write a program to check whether a given number is an ugly number.

Ugly numbers are positive numbers whose prime factors only include 2, 3, 5.

###数论
考察对质因子的理解

In [0]:
class Solution:
    def isUgly(self, num: int) -> bool:
        if num < 1:
            return False
        for i in (2, 3, 5):
            while num % i == 0:
                num = num // i
        if num == 1:
            return True
        return False

##283.move zeroes
Given an array nums, write a function to move all 0's to the end of it while maintaining the relative order of the non-zero elements.

Note:You must do this in-place without making a copy of the array.


###暴力
平方级别的运算复杂度需要优化

在进行类似pop操作时（pop操作其实本身就复制了一遍array），一定注意！步进的条件！进行了pop就不用步进了！不pop的时候才步进！

In [0]:
class Solution:
    def moveZeroes(self, nums) -> None:
        num_cur = 0
        zero_cur = len(nums) - 1
        while num_cur < zero_cur:
            if nums[num_cur] == 0:
                for j in range(num_cur+1,zero_cur+1):
                    nums[j-1] = nums[j]
                nums[zero_cur] = 0
                zero_cur -= 1
            num_cur += 1


###双指针
对于双指针的问题以及原地处理的问题，不要有刻板印象必须要进行嵌套循环，不是必须要每移动一位就全体移位改变一遍。完全可以对每个数从新编号。**一定要摒弃pop思维**

同时，考虑这种问题，不一定要直接考虑是0怎么办，更可以直接从考虑不是0的数怎么办开始入手。

In [0]:
class Solution:
    def moveZeroes(self, nums) -> None:
        pos = 0
        new_index = 0
        while pos < len(nums):
            if nums[pos] != 0:
                nums[new_index] = nums[pos]
                new_index += 1
            pos += 1
        while new_index < len(nums):
            nums[new_index] = 0
            new_index += 1

##349.intersection of two arrays
iven two arrays, write a function to compute their intersection.

Each element in the result must be unique.

The result can be in any order.

###字典&哈希表

In [0]:
class Solution:
    def intersection(self, nums1: List[int], nums2: List[int]) -> List[int]:
        ans = []
        dic = {}
        for i in range(len(nums1)):
            dic[nums1[i]] = 1
        for i in range(len(nums2)):
            if nums2[i] in dic:
                dic[nums2[i]] = 2
        for key,val in dic.items():
            if val == 2:
                ans.append(key)
        return ans
                
        

##371.sum of two integers
Calculate the sum of two integers a and b, but you are not allowed to use the operator + and -.

###位计算
- a^b 得到不含进位之和（均为1时未进位）

- a&b 得到本该进位的位置（均为1的位）
- (a&b)<<1 将这些位左移1，进位到上1位。 但 (a&b)<<1 之后可能仍不为0，则要继续进位。
- 也就是说 a + b = (a ^ b) ^ (carry)

- 只要进位不为零，则迭代；否则返回

In [0]:
    def aplusb(self, a, b):
        fakesum = a ^ b
        carry = (a & b) << 1
        while(carry):
            carry = carry << 1
            ans = fakesum ^ carry 
      
   return ans

#437.@Path Sum III
You are given a binary tree in which each node contains an integer value.

Find the number of paths that sum to a given value.

The path does not need to start or end at the root or a leaf, but it must go downwards (traveling only from parent nodes to child nodes).

The tree has no more than 1,000 nodes and the values are in the range -1,000,000 to 1,000,000.


###双重递归


In [0]:
class Solution:
    
    def pathSum(self, root: TreeNode, sum: int) -> int:
        
        if not root:
            return 0
        return self.dfs(root, sum) + self.pathSum(root.left, sum) + self.pathSum(root.right, sum)
        
    def dfs(self, root, path):
        if not root:
            return 0
        path -= root.val
        return (1 if path==0 else 0) + self.dfs(root.left, path) + self.dfs(root.right, path)

作者：sammy-4
链接：https://leetcode-cn.com/problems/path-sum-iii/solution/hot-100-437lu-jing-zong-he-iii-python3-li-jie-di-g/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

##575.Distribute Candies
Given an integer array with even length, where different numbers in this array represent different kinds of candies. Each number means one candy of the corresponding kind. You need to distribute these candies equally in number to brother and sister. Return the maximum number of kinds of candies the sister could gain.

The length of the given array is in range [2, 10,000], and will be even.

The number in given array is in range [-100,000, 100,000].

###字典&哈希表
要想建立哈希表，则需要遍历整个数组




In [0]:
class Solution:
    def distributeCandies(self, candies: List[int]) -> int:
        hashmap = {}
        n = len(candies)
        for i in range(n):
            hashmap[candies[i]] = hashmap.get(candies[i], 0) + 1

        kinds = len(hashmap)
        if n/2 >= kinds:
            return kinds
        else:
            return n//2

###集合
计算数列中数字种类个数的最快方式：len(set([1,2,3,4]))

In [0]:
class Solution:
    def distributeCandies(self, candies: List[int]) -> int:
        return min(len(set(candies)), len(candies) // 2)

##874.Walking Robot Simulation
A robot on an infinite grid starts at point (0, 0) and faces north.  The robot can receive one of three possible types of commands:

-2: turn left 90 degrees
-1: turn right 90 degrees
1 <= x <= 9: move forward x units
Some of the grid squares are obstacles. 

The i-th obstacle is at grid point (obstacles[i][0], obstacles[i][1])

If the robot would try to move onto them, the robot stays on the previous grid square instead (but still continues following the rest of the route.)

Return the square of the maximum Euclidean distance that the robot will be from the origin.

###复制的
map(func,[])意思就是对[]内每个元素都进行func

In [0]:
class Solution:
    def robotSim(self, commands: List[int], obstacles: List[List[int]]) -> int:
        pos = [0, 0]
        deg = 90
        ans = 0
        obstaclesSet = set(map(tuple, obstacles))#把队列的队列List[List[]]变为元组的集合Set(tuple())

        for command in commands:
            if command == -1:
                deg = (deg + 270) % 360
            elif command == -2:
                deg = (deg + 90) % 360
            else:
                if deg == 0:
                    i = 0
                    while i < command and not (pos[0] + 1, pos[1]) in obstaclesSet:
                        pos[0] += 1
                        i += 1
                if deg == 90:
                    i = 0
                    while i < command and not (pos[0], pos[1] + 1) in obstaclesSet:
                        pos[1] += 1
                        i += 1
                if deg == 180:
                    i = 0
                    while i < command and not (pos[0] - 1, pos[1]) in obstaclesSet:
                        pos[0] -= 1
                        i += 1
                if deg == 270:
                    i = 0
                    while i < command and not (pos[0], pos[1] - 1) in obstaclesSet:
                        pos[1] -= 1
                        i += 1
                ans = max(ans, pos[0] ** 2 + pos[1] ** 2)
        return ans

#Medium

#@46

##49.Group Anagrams
Given an array of strings, group anagrams together.

###字典&哈希表

In [0]:
class Solution:
    def groupAnagrams(self, strs: List[str]) -> List[List[str]]:
        dict = {}

        for str in strs:
            temp = str
            a = list(str)
            a.sort()####一定要注意sort函数没有返回值，是在原地修改的     ####排序一定是对list、tuple等有序队列进行的
            a = "".join(a)
            if a not in dict:####字典的key不能是list
                dict[a] = []
                
            dict[a].append(temp) #sort:str
        print(dict)
        ans = []

        for key, val in dict.items():
            ans.append(val)

        return ans
            
            
        

Python中通过Key访问字典，当Key不存在时，会引发‘KeyError’异常。为了避免这种情况的发生，可以使用collections类中的defaultdict()方法来为字典提供默认值。

语法格式：
collections.defaultdict([default_factory[, …]])

该函数返回一个类似字典的对象。defaultdict是Python内建字典类（dict）的一个子类，它重写了方法_missing_(key)，增加了一个可写的实例变量default_factory,实例变量default_factory被missing()方法使用，如果该变量存在，则用以初始化构造器，如果没有，则为None。其它的功能和dict一样。


In [0]:
class Solution(object):
    def groupAnagrams(self, strs):
        ans = collections.defaultdict(list)
        for s in strs:
            ans[tuple(sorted(s))].append(s) ####tuple被排序后可直接作为字典的key
        return ans.values()

##@55.Jump Game

###动态规划
典型的存在型动态规划问题，这道题目是一道典型的回溯类型题目。 思路就是用一个变量记录当前能够到达的最大的索引，我们逐个遍历数组中的元素去更新这个索引。 变量完成判断这个索引是否大于数组下表即可。

##62.Unique Paths
A robot is located at the top-left corner of a m x n grid (marked 'Start' in the diagram below).  

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid (marked 'Finish' in the diagram below).  

How many possible unique paths are there?

###动态规划
设dp[i, j]表示从(0, 0)到(i,  j)的路径数  
dp[i, j] = dp[i-1, j] + dp[i, j-1]  
dp[0, j] = dp[i, 0] = 1

In [0]:
class Solution:
    def uniquePaths(self, m: int, n: int) -> int:
        dp = [[0] * n for i in range(m)] #常用！创建二维数组的方法
        for i in range(m):
            dp[i][0] = 1
        for j in range(n):
            dp[0][j] = 1
        for i in range(1, m):
            for j in range(1, n):
                dp[i][j] = dp[i-1][j] + dp[i][j-1]
        return dp[m-1][n-1]

##63.Unique Paths II
A robot is located at the top-left corner of a m x n grid (marked 'Start' in the diagram below).

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid (marked 'Finish' in the diagram below).

Now consider if some obstacles are added to the grids. How many unique paths would there be?

###动态规划
难点在于在同一张表上，逐步用路径数替代障碍物状态。  
逻辑：对于初始化边界条件，只有当本格无障碍且前格有一条路径（是通的），本格才有一条路径

In [0]:
class Solution(object):
    def uniquePathsWithObstacles(self, obstacleGrid):
        m = len(obstacleGrid) #常见！求二元数组维数的方法
        n = len(obstacleGrid[0])
        if obstacleGrid[0][0] == 1:
            return 0
        else:
            obstacleGrid[0][0] = 1

        for i in range(1,m):
            obstacleGrid[i][0] = int(obstacleGrid[i][0] == 0 and obstacleGrid[i-1][0] == 1) #只有当本格无障碍且前格有一条路径（是通的），本格才有一条路径
        
        for j in range(1, n):
            obstacleGrid[0][j] = int(obstacleGrid[0][j] == 0 and obstacleGrid[0][j-1] == 1)

        for i in range(1,m):
            for j in range(1,n):
                if obstacleGrid[i][j] == 0:
                    obstacleGrid[i][j] = obstacleGrid[i-1][j] + obstacleGrid[i][j-1]
                else:
                    obstacleGrid[i][j] = 0           
        return obstacleGrid[m-1][n-1]

##@64.Minimum Path Sum
Given a m x n grid filled with non-negative numbers, find a path from top left to bottom right which minimizes the sum of all numbers along its path.

Note: You can only move either down or right at any point in time.

###动态规划


##120.Triangle
Given a triangle, find the minimum path sum from top to bottom. Each step you may move to adjacent numbers on the row below.  
$[
     [2],
    [3,4],
   [6,5,7],
  [4,1,8,3]
]$

###动态规划
  
$dp[i][j] = min(dp[i-1][j-1], dp[i-1][j]) + triangle[i][j]$

In [0]:
class Solution:
    def minimumTotal(self, triangle) -> int:
        row = len(triangle)
        dp = [[0] * row for i in range(row)]
        #initial
        dp[0][0] = triangle[0][0]
        #boader
        for i in range(1, row):
            dp[i][0] = dp[i-1][0] + triangle[i][0]
            dp[i][i] = dp[i-1][i-1] + triangle[i][-1]
        #equotion
        for i in range(2, row):
            for j in range(1, i):
                dp[i][j] = min(dp[i-1][j-1], dp[i-1][j]) + triangle[i][j]
        return min(dp[-1])
    

## @322.Coin Change
You are given coins of different denominations and a total amount of money amount. Write a function to compute the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1.


###动态规划
递归的弊端在于大量的重复计算，比如f(27-7)和f(27-5-2)重复计算，需要使用动态规划需要什么计算什么。
这是典型的最值性动态规划题目  
1.最后一步/子问题：最优策略拿出的最后一枚硬币价值位$a_k$，只需研究$a-a_k$的最优策略  
2.转移方程：f(27)=min{f(27-2)+1, f(27-5)+1, f(27-7)+1}  
3.初始条件和边界条件：f(0)=0，不能拼出时返回正无穷  
4.计算顺序：从小到大



##560.subarray sum equals k
Given an array of integers and an integer k, you need to find the total number of continuous subarrays whose sum equals to k.


###暴力

最暴力的方法是两遍遍历0到n的各种和(cubic)，优化版暴力法是减少重复计算(square)，以后直接从优化版写起

In [0]:
class Solution:
    def subarraySum(self, nums: List[int], k: int) -> int:
        n = len(nums)
        count = 0
        for i in range(n):
            sum = 0
            for j in range(i, n):
                sum += nums[j]
                if sum == k:
                    count += 1
        return count


###！前项和&哈希表
考虑到连续子序列和，必然想到前项和相减  

构建前项和后，问题转化为：当i < j时，看有多少对< i , j >,使得presum[ j ] - presum[ i ] = k，也就是说 presum[ j ] - k  = presum[ i ]

注意：对于长度为n的序列，前项和长度为n+1，因为必须存在0这一项作为减数的情况（即初始化时，直接把0初始化进去）  

look up count -> hash map  

构建字典<presum，frequency>, 每当遍历到一个值，如果他的前序和等于在他之前的字典里的presum - k，即符合条件

In [0]:
class Solution:
    def subarraySum(self, nums: List[int], k: int) -> int:
        n = len(nums)
        presum = 0
        count = 0
        map = {0: 1}   #！重要！map比nums多一位，即0位，如果之后的遍历中存在presum = k，就起码要加1（整个前序和不删减的情况）

        for i in range(n):
            presum += nums[i]

            count += map.get(presum - k, 0) #注意，比较的时候比较的是presum - k(右面的前序和-k，看等于哪个左边的前序和)
            map[presum] = map.get(presum, 0) + 1 #注意，存储的时候存储的是presum
        return count


#counting

In [0]:
class Solution:
    def countElements(self, arr: List[int]) -> int:
        dict = {}
        count = 0
        for num in arr:
            dict[num + 1] = dict.get(num + 1, 0) + 1
        for key, val in dict.items():
            if key in arr:
                count += val
        return count